In [11]:
import numpy as np
from pathlib import Path
import re
from tqdm import tqdm
from skimage import io
from skimage.morphology import skeletonize_3d, binary_dilation, binary_closing
from scipy.ndimage import distance_transform_edt
import tifffile as tif
from scipy.ndimage import binary_fill_holes 
import cc3d
from scipy.io import loadmat, savemat
import skan
import sknw
import networkx as nx
import pickle

In [2]:
#!pip install pystackreg

In [3]:
def remove_small_comps_3d(image, thresh = 1000):
    """
    

    Parameters
    ----------
    image : binary np array with uint8 elements
        3d numpy matrix, connected components will be removed form this image
    thresh : int64
        smallest connected components to keep

    Returns
    -------
    np.array with uint8 elements, binary
        binary image with connected components below the threshold removed.

    """
    img_lab, N = cc3d.connected_components(image,return_N=True)
    unique, counts = np.unique(img_lab, return_counts=True)
    unique_keep = unique[counts>thresh]
    unique_keep = np.delete(unique_keep,[0])
    img_filt = np.zeros(img_lab.shape).astype('int8')
    img_filt[np.isin(img_lab,unique_keep)] = 1
    return img_filt.astype('uint8')   

def fill_holes(img):
    #res = np.zeros(img.shape)
    for i in np.unique(img)[::-1]:
        _tmp = (img==i)*1.0
        _tmp = _tmp.astype('int8')
        _tmp = remove_small_comps_3d(_tmp)
        img[_tmp==1] = i
    res = img.astype('int8')
    return res

In [4]:
directory = Path('james_preds')
files  = directory.glob('*_seg_warped.tif')
files = sorted([x.as_posix() for x in files])
print(len(files))

94


In [23]:
min_prob = 0.75
max_var = 0.1
for file in tqdm(files):
    seg_0001 = io.imread(file)
    #print(file)
    seg = seg_0001.astype('int8')
    print(np.sum(seg))
    #seg = io.imread(re.sub('_0001','',file))
    #print(re.sub('_0001','',file))
    #seg = seg.astype('int8')
    #print(np.sum(seg))
    #seg = seg * seg_0001
    #seg = (seg==1)*1
    #print(np.sum(seg))
    #seg = seg.astype('int8')
    seg = fill_holes(seg)
    #np.save(re.sub('mean','seg',file),seg)
    #savemat(re.sub('_seg_warped.tif','_seg_warped_single.mat',file),{'FinalImage':fill_holes(binary_dilation(binary_dilation(seg)))})
    
    tif.imwrite(re.sub('_seg_warped.tif','_seg_warped_filled.tif',file),seg)
    #print(re.sub('_seg_warped.tif','_seg_warped_single.mat',file))

  0%|          | 0/94 [00:00<?, ?it/s]

951671


  1%|          | 1/94 [00:08<12:38,  8.15s/it]

1644693


  2%|▏         | 2/94 [00:14<10:56,  7.14s/it]

...

 98%|█████████▊| 92/94 [11:07<00:13,  6.89s/it]

2082492


 99%|█████████▉| 93/94 [11:13<00:06,  6.69s/it]

1993329


100%|██████████| 94/94 [11:20<00:00,  7.24s/it]


In [5]:
directory = Path('james_preds')
files  = directory.glob('*_0001_seg_warped.tif')
files = sorted([x.as_posix() for x in files])

In [35]:
files[0]

'james_preds/XYZres006_0001_seg_warped.tif'

In [37]:
re.sub('_seg_warped.tif','_seg_warped_single_SSKKEELL.mat',file)

'james_preds/XYZres006_0001_seg_warped_single_SSKKEELL.mat'

In [8]:
min_prob = 0.75
max_var = 0.1
for file in tqdm(files):
    #break
    seg_0001 = io.imread(file)
    #print(file)
    seg_0001 = seg_0001.astype('int8')
    #print(np.sum(seg_0001))
    seg = io.imread(re.sub('_0001','',file))
    #print(re.sub('_0001','',file))
    seg = seg.astype('int8')
    #print(np.sum(seg))
    seg = seg * seg_0001
    seg = (seg==1)*1
    #print(np.sum(seg))
    seg = seg.astype('int8')
    seg = fill_holes(seg)
    #np.save(re.sub('mean','seg',file),seg)
    savemat(re.sub('_seg_warped.tif','_seg_warped_single.mat',file),{'FinalImage':fill_holes(binary_dilation(seg))})
    print(re.sub('_seg_warped.tif','_seg_warped_single.mat',file))
    #tif.imwrite(re.sub('_seg_warped.tif','_seg_warped_filled_single.tif',file),seg)
    #print(re.sub('_seg_warped.tif','_seg_warped_single.mat',file))

  2%|▏         | 1/47 [00:13<10:09, 13.25s/it]

james_preds/XYZres006_0001_seg_warped_single.mat


  4%|▍         | 2/47 [00:28<10:36, 14.14s/it]

james_preds/XYZres007_0001_seg_warped_single.mat


  6%|▋         | 3/47 [00:43<10:58, 14.96s/it]

...

james_preds/XYZres050_0001_seg_warped_single.mat


 98%|█████████▊| 46/47 [11:11<00:14, 14.49s/it]

james_preds/XYZres051_0001_seg_warped_single.mat


100%|██████████| 47/47 [11:24<00:00, 14.57s/it]

james_preds/XYZres052_0001_seg_warped_single.mat


In [13]:
files_seg_0001 = directory.glob('*_seg_warped_filled.tif')
files_seg_0001 = sorted([x.as_posix() for x in files_seg_0001])
len(files_seg_0001)

94

In [14]:
for file_0001 in tqdm(files_seg_0001[2:]):
    file = file_0001
    print(file)
    skel_file = re.sub('_0001','',re.sub('_seg_warped_filled.tif','_skel_warped_single.mat',file))
    #GETRIDOFNECTLINEINMORNINGIFTOOMANYHAIRSANDRUNMATLAB
    #skel_file = re.sub('_0001_0001','_0001',re.sub('_seg_warped_filled.tif','_0001_seg_warped_single_SSKKEELL.mat',file))
    #
    print(skel_file)
    seg = io.imread(file)
    ###skel = loadmat(skel_file)['FilteredImage']
    #
    skel = loadmat(skel_file)['FilteredImage']
    #
    #skel = skeletonize_3d(skel)
    #skel = remove_small_comps_3d(skel,10)
    #print(np.sum(skel))
    dst_tsfm = distance_transform_edt(seg)
    #dst_tsfm[dst_tsfm==0]=0.5
    skel_dst = skel#*dst_tsfm

    
    np.save(re.sub('seg','dst_skel',file),skel_dst)
    io.imsave(re.sub('_seg_warped_filled.tif','_dst_seg_warped_single.tif',file),dst_tsfm)
    #print(re.sub('_seg_warped_filled.tif','_dst_skel_warped_single.tif',file))
    graph = sknw.build_sknw(skel_dst, multi=False)
    #print(len(graph.edges))
    
    #print(len(graph_0001.edges))
    #pickle.dump(graph, open(str(re.sub('_seg.npy','.pickle',file)), 'w'))
    #nx.write_pajek(graph,re.sub('_seg.npy','.pajek',file))
    nx.write_gpickle(graph,re.sub('_seg_warped_filled.tif','_warped_single.pickle',file))
    print(re.sub('_seg_warped_filled.tif','_warped_single.pickle',file))
    #print(re.sub('_seg_warped_filled.tif','_warped_single.pickle',file))
    #break
    #raph.to_csv(re.sub('seg.npy','graph.csv',file))

  0%|          | 0/92 [00:00<?, ?it/s]

james_preds/XYZres007_0001_seg_warped_filled.tif
james_preds/XYZres007_skel_warped_single.mat


  1%|          | 1/92 [00:22<34:07, 22.50s/it]

james_preds/XYZres007_0001_warped_single.pickle
james_preds/XYZres007_seg_warped_filled.tif
james_preds/XYZres007_skel_warped_single.mat


  2%|▏         | 2/92 [00:33<23:27, 15.64s/it]

...

james_preds/XYZres051_warped_single.pickle
james_preds/XYZres052_0001_seg_warped_filled.tif
james_preds/XYZres052_skel_warped_single.mat


 99%|█████████▉| 91/92 [17:24<00:11, 11.45s/it]

james_preds/XYZres052_0001_warped_single.pickle
james_preds/XYZres052_seg_warped_filled.tif
james_preds/XYZres052_skel_warped_single.mat


100%|██████████| 92/92 [17:36<00:00, 11.48s/it]

james_preds/XYZres052_warped_single.pickle


In [ ]:
np.sum(skel)

In [15]:
graph[0]

AtlasView({172: {'pts': array([[  2, 220, 367],
       [  3, 220, 367],
       [  4, 219, 367],
       [  5, 219, 367],
       [  6, 219, 367],
       [  7, 219, 367],
       [  8, 219, 367],
       [  9, 219, 367],
       [ 10, 219, 367],
       [ 11, 219, 367],
       [ 12, 219, 367],
       [ 13, 218, 368],
       [ 14, 218, 368],
       [ 15, 218, 368],
       [ 16, 218, 368],
       [ 17, 218, 368],
       [ 18, 218, 368],
       [ 19, 218, 368],
       [ 20, 218, 368],
       [ 21, 218, 368],
       [ 22, 217, 369],
       [ 23, 217, 369],
       [ 24, 217, 369],
       [ 25, 217, 369],
       [ 26, 217, 369],
       [ 27, 217, 369],
       [ 28, 217, 369],
       [ 29, 217, 369],
       [ 30, 217, 369],
       [ 31, 217, 369],
       [ 32, 216, 370],
       [ 33, 216, 370],
       [ 34, 216, 369],
       [ 35, 215, 370],
       [ 36, 215, 370],
       [ 37, 215, 370],
       [ 38, 215, 370],
       [ 39, 215, 370],
       [ 40, 215, 370],
       [ 41, 215, 370],
       [ 42, 215

In [ ]:
re.sub('_seg.npy','.pickle',file)

In [ ]:
skel_file = re.sub('seg.npy','skel.mat',file)

In [ ]:
graph[0][39]

In [ ]:
re.sub('seg.npy','graph.csv',file_0001)

In [ ]:
files[22:24]